In [1]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn import metrics
from sklearn.metrics import roc_auc_score

# import matplotlib.pyplot as plt

from sklearn.externals import joblib

In [2]:
import matplotlib.pyplot as plt

In [3]:
from sklearn.metrics import accuracy_score

In [4]:
import pandas as pd
import numpy as np

In [22]:
import pickle

In [5]:
import time

In [14]:
def train_model(train, valid, test, features,target):
    cv_folds = 5
    early_stopping_rounds = 50
    
    ### Forming copy of input datasets
    train_c, valid_c, test_c = train.copy(), valid.copy(), test.copy()
#     out_path = params['out_path']
#     predictors, target = list(params['predictors']), params['target']
    predictors, target = features, target
#     alg = XGBClassifier(
#                 learning_rate =params['learning_rate'],
#                 n_estimators=params['n_estimators'],
#                 max_depth=params['max_depth'],
#                 min_child_weight=params['min_child_weight'],
#                 gamma=params['gamma'],
#                 subsample=params['subsample'],
#                 colsample_bytree=params['colsample_bytree'],
#                 objective= 'binary:logistic',
#                 scale_pos_weight=params['scale_pos_weight'],
#                 seed=27)

    alg = XGBClassifier( colsample_bytree=0.8, gamma=0, learning_rate=0.1,
    max_depth=5, min_child_weight=1, n_estimators=5000, objective='binary:logistic',
    reg_alpha=0, reg_lambda=0, scale_pos_weight=(len(train) -train[df_target].sum())/train[df_target].sum(),
    seed=7, silent=True, subsample=0.8,n_jobs=7)
    
    eval_set = [(valid_c[predictors],valid_c[target])]
    alg.fit(train_c[predictors], train_c[target],eval_metric='auc', early_stopping_rounds=early_stopping_rounds, eval_set = eval_set, verbose = True)     
    
    valid_c.loc[:,'prob'] = alg.predict_proba(valid_c[predictors])[:,1]
    test_c.loc[:,'prob'] = alg.predict_proba(test_c[predictors])[:,1]
    
    valid_auc = roc_auc_score(np.array(valid_c.loc[:,target]), np.array(valid_c.loc[:,'prob']))
    test_auc = roc_auc_score(np.array(test_c.loc[:,target]), np.array(test_c.loc[:,'prob']))
    loss = 1 - valid_auc
    
#     if out_path != None:
#         f = open(os.path.join(out_path, "hyperopt_logs.txt"), "a+")
#         f.write("Valid AUC: {} and Test AUC: {}, loss: {}".format(valid_auc,test_auc,loss))
#         f.close()
    
    print ("Valid AUC: {} and Test AUC: {}, Loss: {}".format(valid_auc, test_auc, loss))
    return alg

In [7]:
def data(df,target):
    processed_df = pd.get_dummies(df,sparse=True)
    df_features = list(set(processed_df.columns) - set([target]))
#     train, valid, test = train_validate_test_split(processed_df)
    train = processed_df.ix[train_o.index]
    valid = processed_df.ix[valid_o.index]
    test = processed_df.ix[test_o.index]
    return train, valid, test, df_features, target

In [8]:
train_o = pd.read_csv('../data/train_insurance.csv',index_col = 0)
valid_o = pd.read_csv('../data/valid_insurance.csv',index_col = 0)
test_o = pd.read_csv('../data/test_insurance.csv',index_col = 0)

In [9]:
all_data = pd.concat([train_o,valid_o,test_o])

In [11]:
all_data.drop(['NewEstimateTotal','EstimateTotal'],axis=1,inplace=True)

In [13]:
train, valid, test, df_features, df_target = data(all_data,target='StartedFlag')

/home/ubuntu/insurance_izenda_analysis/insurace_izenda/lib/python3.5/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
/home/ubuntu/insurance_izenda_analysis/insurace_izenda/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/ubuntu/insurance_izenda_analysis/insurace_izenda/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-

In [18]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=5000,
       n_jobs=7, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=0, scale_pos_weight=0.8181487609316195,
       seed=7, silent=True, subsample=0.8)

In [17]:
start_time = time.time()
model = train_model(train.head(100000), valid.head(20000), test, df_features, df_target)


with open('insurance_classify_100000_started_flag.pkl', 'wb') as f:
    pickle.dump(model, f)

f = open("started_flag_100000.txt", "w")
f.write("Took {} seconds".format(time.time()-start_time))
f.close() 

[0]	validation_0-auc:0.823824
Will train until validation_0-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.828373
[2]	validation_0-auc:0.831332
[3]	validation_0-auc:0.832894
[4]	validation_0-auc:0.834156
[5]	validation_0-auc:0.836691
[6]	validation_0-auc:0.837704
[7]	validation_0-auc:0.838064
[8]	validation_0-auc:0.840287
[9]	validation_0-auc:0.840343
[10]	validation_0-auc:0.840422
[11]	validation_0-auc:0.840475
[12]	validation_0-auc:0.840985
[13]	validation_0-auc:0.84128
[14]	validation_0-auc:0.841887
[15]	validation_0-auc:0.843349
[16]	validation_0-auc:0.843462
[17]	validation_0-auc:0.843732
[18]	validation_0-auc:0.844245
[19]	validation_0-auc:0.844796
[20]	validation_0-auc:0.845435
[21]	validation_0-auc:0.845993
[22]	validation_0-auc:0.84641
[23]	validation_0-auc:0.846736
[24]	validation_0-auc:0.847241
[25]	validation_0-auc:0.84771
[26]	validation_0-auc:0.8481
[27]	validation_0-auc:0.848734
[28]	validation_0-auc:0.849078
[29]	validation_0-auc:0.849735
[30]	validation_0-auc

[259]	validation_0-auc:0.880553
[260]	validation_0-auc:0.88058
[261]	validation_0-auc:0.880596
[262]	validation_0-auc:0.880601
[263]	validation_0-auc:0.880635
[264]	validation_0-auc:0.880704
[265]	validation_0-auc:0.880761
[266]	validation_0-auc:0.880793
[267]	validation_0-auc:0.880878
[268]	validation_0-auc:0.880913
[269]	validation_0-auc:0.880906
[270]	validation_0-auc:0.880924
[271]	validation_0-auc:0.880951
[272]	validation_0-auc:0.880994
[273]	validation_0-auc:0.881032
[274]	validation_0-auc:0.881058
[275]	validation_0-auc:0.881089
[276]	validation_0-auc:0.881115
[277]	validation_0-auc:0.88113
[278]	validation_0-auc:0.881194
[279]	validation_0-auc:0.881378
[280]	validation_0-auc:0.881418
[281]	validation_0-auc:0.88148
[282]	validation_0-auc:0.881519
[283]	validation_0-auc:0.881586
[284]	validation_0-auc:0.881648
[285]	validation_0-auc:0.881677
[286]	validation_0-auc:0.881698
[287]	validation_0-auc:0.881703
[288]	validation_0-auc:0.881895
[289]	validation_0-auc:0.881928
[290]	valid

[516]	validation_0-auc:0.88792
[517]	validation_0-auc:0.887933
[518]	validation_0-auc:0.888007
[519]	validation_0-auc:0.888022
[520]	validation_0-auc:0.888042
[521]	validation_0-auc:0.888087
[522]	validation_0-auc:0.888087
[523]	validation_0-auc:0.888114
[524]	validation_0-auc:0.888147
[525]	validation_0-auc:0.88817
[526]	validation_0-auc:0.888209
[527]	validation_0-auc:0.888233
[528]	validation_0-auc:0.88825
[529]	validation_0-auc:0.888262
[530]	validation_0-auc:0.888289
[531]	validation_0-auc:0.888304
[532]	validation_0-auc:0.888314
[533]	validation_0-auc:0.88833
[534]	validation_0-auc:0.888367
[535]	validation_0-auc:0.888377
[536]	validation_0-auc:0.888409
[537]	validation_0-auc:0.888414
[538]	validation_0-auc:0.888441
[539]	validation_0-auc:0.888468
[540]	validation_0-auc:0.888555
[541]	validation_0-auc:0.888577
[542]	validation_0-auc:0.888614
[543]	validation_0-auc:0.88863
[544]	validation_0-auc:0.888639
[545]	validation_0-auc:0.88865
[546]	validation_0-auc:0.888651
[547]	validati

[773]	validation_0-auc:0.89166
[774]	validation_0-auc:0.891662
[775]	validation_0-auc:0.891669
[776]	validation_0-auc:0.891696
[777]	validation_0-auc:0.891696
[778]	validation_0-auc:0.891724
[779]	validation_0-auc:0.891729
[780]	validation_0-auc:0.89174
[781]	validation_0-auc:0.891741
[782]	validation_0-auc:0.891741
[783]	validation_0-auc:0.89174
[784]	validation_0-auc:0.891742
[785]	validation_0-auc:0.89174
[786]	validation_0-auc:0.891799
[787]	validation_0-auc:0.891809
[788]	validation_0-auc:0.89182
[789]	validation_0-auc:0.891845
[790]	validation_0-auc:0.891845
[791]	validation_0-auc:0.891857
[792]	validation_0-auc:0.891877
[793]	validation_0-auc:0.891877
[794]	validation_0-auc:0.891882
[795]	validation_0-auc:0.891891
[796]	validation_0-auc:0.891914
[797]	validation_0-auc:0.891918
[798]	validation_0-auc:0.891919
[799]	validation_0-auc:0.891928
[800]	validation_0-auc:0.891938
[801]	validation_0-auc:0.891946
[802]	validation_0-auc:0.891941
[803]	validation_0-auc:0.891953
[804]	validat

[1030]	validation_0-auc:0.89382
[1031]	validation_0-auc:0.893828
[1032]	validation_0-auc:0.893831
[1033]	validation_0-auc:0.893836
[1034]	validation_0-auc:0.893847
[1035]	validation_0-auc:0.893864
[1036]	validation_0-auc:0.893903
[1037]	validation_0-auc:0.893912
[1038]	validation_0-auc:0.893922
[1039]	validation_0-auc:0.893918
[1040]	validation_0-auc:0.893924
[1041]	validation_0-auc:0.893918
[1042]	validation_0-auc:0.89395
[1043]	validation_0-auc:0.893945
[1044]	validation_0-auc:0.893983
[1045]	validation_0-auc:0.893984
[1046]	validation_0-auc:0.893989
[1047]	validation_0-auc:0.893989
[1048]	validation_0-auc:0.89399
[1049]	validation_0-auc:0.893992
[1050]	validation_0-auc:0.893989
[1051]	validation_0-auc:0.893992
[1052]	validation_0-auc:0.89399
[1053]	validation_0-auc:0.894004
[1054]	validation_0-auc:0.894008
[1055]	validation_0-auc:0.894015
[1056]	validation_0-auc:0.894017
[1057]	validation_0-auc:0.894014
[1058]	validation_0-auc:0.894024
[1059]	validation_0-auc:0.894035
[1060]	validat

[1280]	validation_0-auc:0.895598
[1281]	validation_0-auc:0.895602
[1282]	validation_0-auc:0.895598
[1283]	validation_0-auc:0.895607
[1284]	validation_0-auc:0.895603
[1285]	validation_0-auc:0.895616
[1286]	validation_0-auc:0.895614
[1287]	validation_0-auc:0.89562
[1288]	validation_0-auc:0.895619
[1289]	validation_0-auc:0.895633
[1290]	validation_0-auc:0.895638
[1291]	validation_0-auc:0.89566
[1292]	validation_0-auc:0.895656
[1293]	validation_0-auc:0.895675
[1294]	validation_0-auc:0.895679
[1295]	validation_0-auc:0.895683
[1296]	validation_0-auc:0.895673
[1297]	validation_0-auc:0.895671
[1298]	validation_0-auc:0.895658
[1299]	validation_0-auc:0.895662
[1300]	validation_0-auc:0.8957
[1301]	validation_0-auc:0.895704
[1302]	validation_0-auc:0.895701
[1303]	validation_0-auc:0.895722
[1304]	validation_0-auc:0.895716
[1305]	validation_0-auc:0.895725
[1306]	validation_0-auc:0.895733
[1307]	validation_0-auc:0.895736
[1308]	validation_0-auc:0.895748
[1309]	validation_0-auc:0.895746
[1310]	validat

[1530]	validation_0-auc:0.896772
[1531]	validation_0-auc:0.896816
[1532]	validation_0-auc:0.896809
[1533]	validation_0-auc:0.896792
[1534]	validation_0-auc:0.896782
[1535]	validation_0-auc:0.896788
[1536]	validation_0-auc:0.896793
[1537]	validation_0-auc:0.89679
[1538]	validation_0-auc:0.896795
[1539]	validation_0-auc:0.896779
[1540]	validation_0-auc:0.896782
[1541]	validation_0-auc:0.896826
[1542]	validation_0-auc:0.896816
[1543]	validation_0-auc:0.89681
[1544]	validation_0-auc:0.896814
[1545]	validation_0-auc:0.89681
[1546]	validation_0-auc:0.896831
[1547]	validation_0-auc:0.896839
[1548]	validation_0-auc:0.89685
[1549]	validation_0-auc:0.896868
[1550]	validation_0-auc:0.896867
[1551]	validation_0-auc:0.896886
[1552]	validation_0-auc:0.896893
[1553]	validation_0-auc:0.896898
[1554]	validation_0-auc:0.896922
[1555]	validation_0-auc:0.896912
[1556]	validation_0-auc:0.896923
[1557]	validation_0-auc:0.896931
[1558]	validation_0-auc:0.896928
[1559]	validation_0-auc:0.896927
[1560]	validat

[1779]	validation_0-auc:0.897724
[1780]	validation_0-auc:0.897732
[1781]	validation_0-auc:0.897741
[1782]	validation_0-auc:0.897735
[1783]	validation_0-auc:0.897752
[1784]	validation_0-auc:0.897749
[1785]	validation_0-auc:0.897758
[1786]	validation_0-auc:0.897772
[1787]	validation_0-auc:0.89778
[1788]	validation_0-auc:0.897788
[1789]	validation_0-auc:0.897797
[1790]	validation_0-auc:0.897795
[1791]	validation_0-auc:0.897806
[1792]	validation_0-auc:0.897814
[1793]	validation_0-auc:0.897817
[1794]	validation_0-auc:0.897829
[1795]	validation_0-auc:0.897825
[1796]	validation_0-auc:0.89783
[1797]	validation_0-auc:0.897844
[1798]	validation_0-auc:0.897839
[1799]	validation_0-auc:0.897843
[1800]	validation_0-auc:0.897842
[1801]	validation_0-auc:0.897837
[1802]	validation_0-auc:0.897849
[1803]	validation_0-auc:0.897843
[1804]	validation_0-auc:0.897846
[1805]	validation_0-auc:0.897852
[1806]	validation_0-auc:0.897846
[1807]	validation_0-auc:0.897852
[1808]	validation_0-auc:0.897848
[1809]	valid

NameError: name 'pickle' is not defined

In [23]:
with open('insurance_classify_100000_started_flag.pkl', 'wb') as f:
    pickle.dump(model, f)

f = open("started_flag_100000.txt", "w")
f.write("Took {} seconds".format(time.time()-start_time))
f.close() 

In [ ]:
# To load the pickle model\n",
with open('insurance_classify_100000_estimate_total.pkl', 'rb') as f:
    model = pickle.load(f)

In [20]:
valid.loc[:,'prob_up'] = model.predict_proba(valid[df_features])[:,1]

valid['prediction'] = np.where(valid.prob_up > 0.5,1,0)

accuracy_score(valid.StartedFlag, valid.prediction)

0.82179240007352

In [21]:
test.loc[:,'prob_up'] = model.predict_proba(test[df_features])[:,1]

test['prediction'] = np.where(test.prob_up > 0.5,1,0)

accuracy_score(test.StartedFlag, test.prediction)

0.8212032927282326

In [25]:
# Confusion Matrix on Test Data
metrics.confusion_matrix(test.StartedFlag, test.prediction)

array([[ 95387,  19929],
       [ 25792, 114607]])